[View in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/tools/GoogleScholar.ipynb)

# Google scholar

Search for similarity in title and one auhor. Returns Python dictionary

__WARNING__: Does not work from Colaboratory

In [0]:
import Levenshtein
import re
from bs4 import BeautifulSoup
import requests

def google_scholar(title='relativity theory',author="A Einstein"):
    '''
    Search Google scholar for similarity in title and author.
    Only the first result is analized. The output includes 
    a quality measurements between the query and the results 
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','cites','cites_link',
    'quality_title','quality_author'
    '''
    # + → %2B in query formula:
    url='https://scholar.google.com/scholar?q="{}"%2B{}'.format(title,author)
    r=requests.get(url)

    soup = BeautifulSoup(r.text, "html.parser")

    rgs=soup.find_all('div', {'class':'gs_ri' })

    gs={}

    try:
        cites=rgs[0].find_all('a',{"href":re.compile( "/scholar\?cites*" )})[0]
        try:
            gs['cites']=int( cites.text.split()[-1] )
            gs['cites_link']=cites.attrs.get('href')
        except:
            gs['cites']=0
    except:
        cites=None

    # Título
    try:
        lstt=rgs[0].find_all('a',{"data-clk":re.compile( ".*")})[0].contents
    except:
        lstt=[]

    # Encontrar title quitando caracteres raros
    tc=''
    for tip in lstt:
        if tip.find('svg')==-1:
            tc=tc+str(tip)
        else:
            tc=tc+tip.get('aria-label') # aria label in Accessibility
    gs['title']=tc

    gpa=None
    try:
        gpa=rgs[0].find_all('div',{"class":"gs_a"})[0]
        gs['authors']=gpa.text.split('-')[0].strip()
    except:
        gs['authors']=''


    if gpa:
        lpr=gpa.find_all("a",{ "href":re.compile("/citations\?user=*")   } )
        prf={}
        for pr in lpr:
            prf[ pr.text ]=pr.attrs.get('href').split('?')[-1].split('&')[0].split('user=')[-1]
        gs['profiles']=prf
        
    sau=0
    for a in gs['authors'].split(','):
        saun=Levenshtein.ratio(author.lower(),a.lower().strip())
        if saun>sau:
            sau=saun
    gs['quality_author']=round(sau,2)
        
    gs['quality_title']=round( Levenshtein.ratio(
               title.lower(),gs['title'].lower() ),2 )

    return gs

In [0]:
google_scholar(title='Fermionic triplet dark matter in an SO(10)-inspired left right model',
              author='D Restrepo')

{'authors': 'C Arbeláez, M Hirsch, D Restrepo',
 'cites': 7,
 'cites_link': '/scholar?cites=10539747452972188249&as_sdt=2005&sciodt=0,5&hl=en&oe=ASCII',
 'profiles': {'D Restrepo': '1sKULCoAAAAJ'},
 'quality_author': 1.0,
 'quality_title': 0.99,
 'title': 'Fermionic triplet dark matter in an SO(10)-inspired left-right model'}